In [2]:
import pandas as pd


In [3]:
data_all = []
for i in range(4):
    data = pd.read_csv('netflix/combined_data_' + str(i+1)+'.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
    data_all.append(data)

MemoryError: Unable to allocate 206. MiB for an array with shape (1, 26982302) and data type object

In [3]:
df = pd.concat(data_all)

In [4]:
import numpy as np
df.index = np.arange(0,len(df))
print('Full dataset shape: {}'.format(df.shape))
print('-Dataset examples-')
print(df.iloc[::5000000, :])

Full dataset shape: (100498277, 2)
-Dataset examples-
           Cust_Id  Rating
0               1:     NaN
5000000    2560324     4.0
10000000   2271935     2.0
15000000   1921803     2.0
20000000   1933327     3.0
25000000   1465002     3.0
30000000    961023     4.0
35000000   1372532     5.0
40000000    854274     5.0
45000000    116334     3.0
50000000    768483     3.0
55000000   1331144     5.0
60000000   1609324     2.0
65000000   1699240     3.0
70000000   1776418     4.0
75000000   1643826     5.0
80000000    932047     4.0
85000000   2292868     4.0
90000000    932191     4.0
95000000   1815101     3.0
100000000   872339     4.0


In [ ]:
df.shape

In [ ]:
import numpy as np
df_nan = pd.DataFrame(pd.isnull(df.Rating))
df_nan = df_nan[df_nan['Rating'] == True]
df_nan = df_nan.reset_index()

movie_np = []
movie_id = 1

for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    # numpy approach
    temp = np.full((1,np.abs(i-j-1)), movie_id)
    movie_np = np.append(movie_np, temp)
    movie_id += 1

# Account for last record and corresponding length
# numpy approach
last_record = np.full((1,len(df) - df_nan.iloc[-1, 0] - 1),movie_id)
movie_np = np.append(movie_np, last_record)

In [ ]:
df = df[pd.notnull(df['Rating'])]

df['Movie_Id'] = movie_np.astype(int)
df['Cust_Id'] = df['Cust_Id'].astype(int)
print('-Dataset examples-')
print(df.sample(3))

In [ ]:
df.to_csv('/srv/aprosvetov/netflix/data_prep.csv', sep=';', index = None)

In [5]:
import pandas as pd
df = pd.read_csv('/srv/aprosvetov/netflix/data_prep.csv', sep=';')

In [6]:
df.head()

,Cust_Id,Rating,Movie_Id
0,1488844,3.0,1
1,822109,5.0,1
2,885013,4.0,1
3,30878,4.0,1
4,823519,3.0,1


In [7]:
good = df[df['Rating']==5].groupby('Cust_Id')['Movie_Id'].apply(lambda r: ' '.join([str(A) for A in r]))

In [8]:
good.head()

Cust_Id
6     175 457 886 1467 2372 2452 2782 3290 4043 4633...
7     8 30 83 175 257 283 285 313 357 457 458 468 50...
8     1202 1799 1905 2186 3610 3925 4306 5054 5317 5...
10    473 985 1542 1905 2172 3124 3371 3962 4043 430...
25                4432 6786 7605 9326 10643 15107 15270
Name: Movie_Id, dtype: object

In [9]:
import apyori

In [23]:
association_rules = apyori.apriori(good.apply(lambda r: r.split(' ')), 
                                   min_support=0.04, 
                                   min_confidence=0.1, min_lift=2, 
                                   min_length=2)

In [11]:
association_rules

<generator object apriori at 0x7fa0a153cd58>

In [24]:
asr_df = pd.DataFrame(columns = ['from', 'to', 'confidence', 'support', 'lift'])
for item in association_rules:
    pair = item[0] 
    items = [x for x in pair]
    asr_df.loc[len(asr_df), :] =  ' '.join(list(item[2][0][0])), \
                                  ' '.join(list(item[2][0][1])),\
                                  item[2][0][2], item[1], item[2][0][3]

In [25]:
asr_df

,from,to,confidence,support,lift
0,10042,10820,0.266215,0.0415732,4.11022
1,10042,10947,0.351478,0.0548881,2.42194
2,10042,11064,0.348135,0.0543661,2.3007
3,10042,11089,0.331616,0.0517864,2.48827
4,10042,11283,0.454613,0.0709941,2.20623
5,10042,11521,0.552044,0.0862093,2.65123
6,10042,11781,0.441243,0.0689062,4.69159
7,10042,12293,0.391395,0.0611217,2.64145
8,10042,12870,0.349931,0.0546465,2.67333
9,10042,13728,0.379793,0.0593099,2.53091


In [26]:
asr_df.sort_values('lift').tail(10)

,from,to,confidence,support,lift
1982,14240 14961 2452 7057 16265,7230,0.957116,0.0453004,7.70559
1999,7057 14961 2452 5582 16265,7230,0.957563,0.0430248,7.70919
2008,14961 11521 7057 2452 14240 16265,7230,0.95882,0.0439437,7.71931
2020,14961 7057 5582 2452 14240 16265,7230,0.95994,0.0417112,7.72833
2018,14961 11521 7057 5582 2452 16265,7230,0.960066,0.0414848,7.72934
2022,14961 11521 7057 5582 2452 14240 16265,7230,0.961517,0.0405271,7.74102
143,11443,17627,0.734164,0.0657225,7.95834
764,11443 11521,17627,0.790981,0.041653,8.57423
766,11443 12338,17627,0.854732,0.0439243,9.2653
313,14302,16147,0.741323,0.0403588,12.7193


In [18]:
titles = pd.read_csv('movie_titles.csv', encoding = "ISO-8859-1", 
                     header = None, 
                     names = ['Movie_Id', 'Year', 'Name'])

In [19]:
def get_rule_title(rule):
    print(titles[titles.Movie_Id.isin(rule['from'].split(' '))]['Name'].values)
    print('----------->')
    print(titles[titles.Movie_Id == int(rule['to'])]['Name'].values)

In [27]:
get_rule_title(asr_df.loc[2022])

['Lord of the Rings: The Fellowship of the Ring'
 'Star Wars: Episode V: The Empire Strikes Back'
 'Lord of the Rings: The Two Towers: Extended Edition'
 'Lord of the Rings: The Two Towers'
 'Lord of the Rings: The Return of the King'
 'Lord of the Rings: The Return of the King: Extended Edition'
 'Star Wars: Episode IV: A New Hope']
----------->
['The Lord of the Rings: The Fellowship of the Ring: Extended Edition']


In [ ]:
get_rule_title(asr_df.sample(1).iloc[0])

In [ ]:
good.iloc[14]

In [28]:
j = 159992



In [29]:
titles[titles.Movie_Id.isin(good.iloc[j].split(' '))]['Name']

1144     The Wedding Planner
2151         What Women Want
5316       Miss Congeniality
6286            Pretty Woman
6385              Sister Act
7233            Men of Honor
10358          Runaway Bride
11148      Maid in Manhattan
13049       Two Weeks Notice
15581     Sweet Home Alabama
Name: Name, dtype: object

In [30]:
def print_rule_title(rule):
    return (titles[titles.Movie_Id == int(rule['to'])]['Name'].values)
    

result = []
for A in asr_df.index:
    if len(set(good.iloc[j].split(' ')) & set(asr_df['from'].loc[A].split(' '))) == len(asr_df['from'].loc[A].split(' ')):
        result.append(print_rule_title(asr_df.loc[A])[0])
print(set(result))

{'Dirty Dancing', 'Pretty Woman'}
